In [1]:
import pandas as pd
from tqdm.notebook import tqdm_notebook as tqdm
import os
import itertools
from rsa import compute_rsa
import gc
import json

In [2]:
with open('../../data/raw/dtype_to_embed.json') as f:
    dtype_to_embed = json.load(f)

names = dtype_to_embed['text'] + dtype_to_embed['behavior'] + dtype_to_embed['brain']
name_combs = list(itertools.combinations(names, 2))
len(name_combs)

300

In [3]:
# Function to process each file pair
def process_file_pair(f_name_i, f_name_j, dir_path):
    
    # Load the RSMs
    rsm_i = pd.read_pickle(dir_path + f_name_i)
    rsm_j = pd.read_pickle(dir_path + f_name_j)

    # Compute RSA
    corr, n_words = compute_rsa(rsm_i, rsm_j, max_n=10000)
    print(f"spearman_r={corr}, n_words={n_words}")
    print('-------------------------------------------------')
    
    # Free memory by deleting the RSMs
    del rsm_i, rsm_j
    gc.collect()

    return corr, n_words

# Make the output directory
rsm_dir_path = f'../../data/processed/rsms/'
output_dir_path = '../../data/final/'
if not os.path.exists(output_dir_path):
    os.makedirs(output_dir_path)

# Compute RSA in parallel
rsa = []
for name_i, name_j in tqdm(name_combs):
    print(f"{name_i, name_j}")
    spearman, n = process_file_pair(f'{name_i}.pkl', f'{name_j}.pkl', rsm_dir_path)
    rsa.append([name_i, name_j, spearman, n])
    
# Save the results
rsa_df = pd.DataFrame(rsa, columns=['name_i', 'name_j', 'spearman', 'n_words'])
rsa_df.to_csv(f'../../data/final/rsa.csv', index=False)

  0%|          | 0/300 [00:00<?, ?it/s]

('CBOW_GoogleNews', 'fastText_CommonCrawl')
(42522, 42522)
(44091, 44091)
Reduced to 500 words
Aligned RSMs
Converted RSM i to numpy
Converted RSM j to numpy
Filled self-correlations with nan
spearman_r=0.6291632236109945, n_words=42374
-------------------------------------------------
('CBOW_GoogleNews', 'fastText_Wiki_News')
(42522, 42522)
(42792, 42792)
Reduced to 500 words
Aligned RSMs
Converted RSM i to numpy
Converted RSM j to numpy
Filled self-correlations with nan
spearman_r=0.5432315493857449, n_words=41950
-------------------------------------------------
('CBOW_GoogleNews', 'fastTextSub_OpenSub')
(42522, 42522)
(40264, 40264)
Reduced to 500 words
Aligned RSMs
Converted RSM i to numpy
Converted RSM j to numpy
Filled self-correlations with nan
spearman_r=0.4631967657959426, n_words=39759
-------------------------------------------------
('CBOW_GoogleNews', 'GloVe_CommonCrawl')
(42522, 42522)
(43927, 43927)
Reduced to 500 words


KeyboardInterrupt: 